In [1]:
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix
import numpy as np 
import torch
import dgl
from dgl.data import AsNodePredDataset
from ogb.nodeproppred import DglNodePropPredDataset

/home/bear/miniconda3/envs/dgl_v09/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
graph = np.fromfile("/home/bear/workspace/single-gnn/data/partition/PD/part0/raw_G.bin",dtype=np.int32)
nodes = np.fromfile("/home/bear/workspace/single-gnn/data/partition/PD/part0/raw_nodes.bin",dtype=np.int32)
trainIds = np.fromfile("/home/bear/workspace/single-gnn/data/partition/PD/part0/raw_trainIds.bin",dtype=np.int64)

In [18]:
graph = graph.reshape(-1)

In [19]:
src = torch.tensor(graph[::2])
dst = torch.tensor(graph[1::2])
nodeTable = torch.tensor(nodes)
uni = torch.zeros(len(nodeTable)*2).to(torch.int32)

In [21]:
dst

tensor([      0,       1,       2,  ...,  162836, 2449028,   53324],
       dtype=torch.int32)

In [22]:
shuffle_src,shuffle_dst,uni = dgl.mapByNodeSet(nodeTable.cuda(),uni.cuda(),src.cuda(),dst.cuda())

In [29]:
uni[53285]

tensor(53324, device='cuda:0', dtype=torch.int32)

In [25]:
src

tensor([      0,       1,       2,  ..., 2449028,   53324, 2449028],
       dtype=torch.int32)

In [26]:
shuffle_src

tensor([      0,       1,       2,  ..., 2216129,   53285, 2216129],
       device='cuda:0', dtype=torch.int32)

In [27]:
shuffle_dst

tensor([      0,       1,       2,  ...,  162645, 2216129,   53285],
       device='cuda:0', dtype=torch.int32)